In [1]:
import pandas as pd
import numpy as np
import glob
import os
import pickle

startDate = '20190729'
endDate = '20190927'

readPath = r'K:\recordData\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

F1 = open("\\\\192.168.10.30\\Kevin_zhenyu\\index\\weight_table_IC.pkl", 'rb')
df1 = pickle.load(F1)

for i in dataPathLs:
    path = r'\\192.168.10.30\Kevin_zhenyu\record data' 
    os.mkdir(path + '\\' + os.path.basename(i).split('_')[1])
    
    logSZ = pd.read_csv(np.array(glob.glob(i + "\\mdLog_SZ***"))[0],
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    mdOrderLog = pd.read_csv(np.array(glob.glob(i + "\\mdOrderLog***"))[0],
                    encoding="utf-8").loc[:, ['clockAtArrival', 'TransactTime', 'sequenceNo', 'SecurityID', 'ApplSeqNum', 
                                              'Side', 'OrderType', 'Price', 'OrderQty']]
    mdTradeLog = pd.read_csv(np.array(glob.glob(i + "\\mdTradeLog***"))[0],
                    encoding="utf-8").loc[:, ['clockAtArrival', 'TransactTime', 'sequenceNo', 'SecurityID', "exchId", 'ApplSeqNum', 
                                              'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 
                                              'OfferApplSeqNum']]
    logSH = pd.read_csv(np.array(glob.glob(i + "\\mdLog_SH***"))[0],
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    in_dex = [16, 300, 852, 905]
    logSH = logSH[logSH["StockID"].isin(in_dex)]
    logSZ["newID"] = logSZ["exchange"] + logSZ["StockID"].astype(str).apply(lambda x: x.rjust(6, '0'))
    da_te = os.path.basename(i).split('_')[1]
    da_te = da_te[:4] + '-' + da_te[4:6] + '-' + da_te[6:]
    logSZ = logSZ[(logSZ["newID"].isin(df1[df1["date"] == da_te]["ID"].unique())) & (logSZ["source"] == 4)]
    logSZ["ID"] = logSZ["StockID"] + 2000000
    logSH["ID"] = logSH["StockID"] + 1000000
    mdData = pd.concat([logSH, logSZ])
    mdData['time'] = mdData.time.str.slice(0, 2) + mdData.time.str.slice(3, 5) + mdData.time.str.slice(6, 8) + '000'
    mdData['time'] = mdData['time'].astype('int64')
    mdData['time'] = mdData.groupby(['ID'])['time'].cummax()
    mdData = mdData[(mdData["cum_volume"] > 0) & (mdData["time"] <= 150000000)]
    mdData = mdData.sort_values(by="sequenceNo").reset_index(drop=True)
    mdDataCols = ['clockAtArrival', 'time', 'sequenceNo', 'ID', 'cum_volume', 'cum_amount', 'close',
              'bid1p', 'bid2p', 'bid3p', 'bid4p', 'bid5p', 'bid1q', 'bid2q', 'bid3q', 'bid4q', 'bid5q',
              'ask1p', 'ask2p', 'ask3p', 'ask4p', 'ask5p', 'ask1q', 'ask2q', 'ask3q', 'ask4q', 'ask5q', 'openPrice']
    mdData = mdData[mdDataCols]
    display(mdData.head(5))
    mdData.to_pickle(r"\\192.168.10.30\Kevin_zhenyu\record data" + '\\' + os.path.basename(i).split('_')[1] + '\\'
                    + 'mdLog_' + os.path.basename(i).split('_')[1] + '.pkl')
    
    mdOrderLog["newID"] = 'SZ' + mdOrderLog["SecurityID"].astype(str).apply(lambda x: x.rjust(6, '0'))
    mdOrderLog = mdOrderLog[mdOrderLog["newID"].isin(df1[df1["date"] == da_te]["ID"].unique())]
    mdOrderLog = mdOrderLog.rename(columns={"TransactTime": "time"})
    mdOrderLog["ID"] = mdOrderLog["SecurityID"] + 2000000
    mdOrderLog["OrderType"] = mdOrderLog["OrderType"].astype(str)
    mdOrderLogCols = ['clockAtArrival', 'time', 'sequenceNo', 'ID', 'ApplSeqNum', 'Side', 'OrderType', 'Price', 'OrderQty']
    mdOrderLog = mdOrderLog[mdOrderLogCols]
    display(mdOrderLog.head(5))
    mdOrderLog.to_pickle(r"\\192.168.10.30\Kevin_zhenyu\record data" + '\\' + os.path.basename(i).split('_')[1] + '\\'
                    + 'mdOrderLog_' + os.path.basename(i).split('_')[1] + '.pkl')
    
    mdTradeLog = mdTradeLog[mdTradeLog["exchId"] != 1]
    mdTradeLog["newID"] = 'SZ' + mdTradeLog["SecurityID"].astype(str).apply(lambda x: x.rjust(6, '0'))
    mdTradeLog = mdTradeLog[mdTradeLog["newID"].isin(df1[df1["date"] == da_te]["ID"].unique())]
    mdTradeLog = mdTradeLog.rename(columns={"TransactTime": "time"})
    mdTradeLog["ID"] = mdTradeLog["SecurityID"] + 2000000
    mdTradeLogCols = ['clockAtArrival', 'time', 'sequenceNo', 'ID', 'ApplSeqNum', 'ExecType', 
                      'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']
    mdTradeLog = mdTradeLog[mdTradeLogCols]
    display(mdTradeLog.head(5))
    mdTradeLog.to_pickle(r"\\192.168.10.30\Kevin_zhenyu\record data" + '\\' + os.path.basename(i).split('_')[1] + '\\'
                    + 'mdTradeLog_' + os.path.basename(i).split('_')[1] + '.pkl')
    


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,clockAtArrival,time,sequenceNo,ID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,...,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
0,1564536318884921,92503000,1423377,2000008,24000,86880.00,3.62,3.61,3.60,3.59,...,3.63,3.64,3.65,3.66,76300,114300,108900,279100,209024,3.62
1,1564536318884978,92503000,1423379,2000012,8200,34440.00,4.20,4.20,4.19,4.18,...,4.23,4.24,4.25,4.26,47286,7000,97730,35700,16100,4.20
2,1564536318885014,92503000,1423383,2000028,3100,127844.00,41.24,41.12,41.06,41.04,...,41.26,41.29,41.32,41.33,100,800,2200,1200,500,41.24
3,1564536318885130,92503000,1423385,2000031,90516,639948.12,7.07,7.07,7.06,7.05,...,7.10,7.11,7.12,7.15,12900,25816,2000,2000,16000,7.07
4,1564536318885244,92503000,1423391,2000060,53700,235206.00,4.38,4.37,4.36,4.35,...,4.39,4.40,4.41,4.42,83500,51100,163400,133700,68600,4.38


,clockAtArrival,time,sequenceNo,ID,ApplSeqNum,Side,OrderType,Price,OrderQty
168,1564535715811929,91500000,293949,2002517,169,1,2,31600,100
331,1564535715814105,91500000,294112,2002131,312,2,2,18000,553700
373,1564535715814154,91500000,294154,2002920,355,1,2,259100,300
389,1564535715814176,91500000,294170,2002517,371,1,2,34800,53100
528,1564535715815360,91500000,294309,2002407,386,2,2,151100,7500


,clockAtArrival,time,sequenceNo,ID,ApplSeqNum,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
11,1564535715841956,91500020,298416,2002157,940,4,0,8000,0,0,915
36,1564535716087552,91500130,316250,2002482,5800,4,0,1200,0,5799,0
37,1564535716118071,91500140,318749,2300027,6449,4,0,7000,0,6235,0
49,1564535716351450,91500230,335676,2002372,11246,4,0,100,0,0,11244
54,1564535716409254,91500240,340372,2000536,12258,4,0,4200,0,0,12239


,clockAtArrival,time,sequenceNo,ID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,...,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
0,1564363518568228,92503000,1321217,2000006,1100,5885.0,5.35,5.35,5.33,5.32,...,5.39,5.40,5.41,5.42,2900,2700,18400,5100,6900,5.35
1,1564363518568239,92503000,1321219,2000012,7800,32994.0,4.23,4.23,4.22,4.21,...,4.26,4.27,4.28,4.29,15900,34600,9400,35636,56080,4.23
2,1564363518568321,92503000,1321224,2000027,30200,178180.0,5.90,5.90,5.89,5.88,...,5.93,5.94,5.95,5.96,23500,2100,2100,34700,9500,5.90
3,1564363518568630,92503000,1321234,2000078,108200,361388.0,3.34,3.34,3.33,3.32,...,3.36,3.37,3.38,3.39,40000,72300,12800,9000,10200,3.34
4,1564363518568660,92503000,1321238,2000156,26000,273260.0,10.51,10.50,10.48,10.45,...,10.52,10.53,10.54,10.55,24600,15000,7200,11300,31800,10.51


...

,clockAtArrival,time,sequenceNo,ID,ApplSeqNum,Side,OrderType,Price,OrderQty
185,1569460499377009,91500000,324447,2002589,186,1,2,84700,5000
291,1569460499377923,91500000,324553,2000975,293,1,2,174900,102800
323,1569460499378245,91500000,324585,2000975,325,1,2,174900,114500
331,1569460499378330,91500000,324593,2000975,333,1,2,174900,11600
339,1569460499378397,91500000,324601,2000975,341,1,2,174900,171300


,clockAtArrival,time,sequenceNo,ID,ApplSeqNum,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
9,1569460499443275,91500030,336248,2002373,2248,4,0,16800,0,0,2242
10,1569460499443276,91500030,336249,2002373,2249,4,0,51200,0,0,2245
11,1569460499443277,91500030,336250,2002373,2250,4,0,15500,0,0,2246
12,1569460499443278,91500030,336251,2002373,2251,4,0,9800,0,0,2247
13,1569460499443279,91500030,336252,2002373,2252,4,0,21600,0,0,2241


,clockAtArrival,time,sequenceNo,ID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,...,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
0,1569547502709591,92503000,1370135,2000031,41366,277565.86,6.71,6.71,6.70,6.68,...,6.78,6.79,6.80,6.81,600,1000,1000,10200,2200,6.71
1,1569547502720358,92503000,1370164,2000027,117000,684450.00,5.85,5.84,5.83,5.82,...,5.94,5.95,5.96,5.97,23500,5500,1000,5000,3000,5.85
2,1569547502720450,92503000,1370168,2000060,64900,262196.00,4.04,4.04,4.03,4.02,...,4.06,4.07,4.08,4.09,2850,367700,51830,96200,233200,4.04
3,1569547502720458,92503000,1370169,2000061,3300,17259.00,5.23,5.19,5.18,5.17,...,5.25,5.26,5.27,5.28,1500,11000,4000,8000,40500,5.23
4,1569547502720462,92503000,1370170,2000062,41227,606449.17,14.71,14.71,14.61,14.55,...,14.74,14.75,14.76,14.77,20900,600,2000,6600,15100,14.71


,clockAtArrival,time,sequenceNo,ID,ApplSeqNum,Side,OrderType,Price,OrderQty
19,1569546899577497,91500000,279144,2300115,21,2,2,149800,6300
27,1569546899577615,91500000,279152,2000975,29,2,2,128800,7700
39,1569546899577727,91500000,279164,2000878,41,2,2,101500,1000
45,1569546899577742,91500000,279170,2000021,47,2,2,117000,400
56,1569546899578171,91500000,279181,2000975,57,2,2,151000,9000


,clockAtArrival,time,sequenceNo,ID,ApplSeqNum,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
25,1569546899695363,91500090,299272,2000723,5148,4,0,500,0,5147,0
31,1569546899793922,91500180,315668,2002281,9490,4,0,2000,0,9112,0
35,1569546899806579,91500180,317815,2000158,9977,4,0,25900,0,0,4766
38,1569546899824589,91500200,320060,2000547,10152,4,0,300000,0,10151,0
42,1569546899862155,91500230,326632,2002440,11581,4,0,6800,0,0,11580
